# Questions:
1. Which job on average pays the most?
2. Which job has the most capital?
3. Which job has the most employed people?
4. What is the difference in average salary based on each employment type?
5. Do companies tend to hire employees residing in the same country?
6. Is there a difference in pay between employees who got hired in the same year, and have the same job?

In [22]:
# Importing the libraries
import pandas as pd
import plotly.express as px

In [23]:
# Importing the dataset
df = pd.read_csv("salaries.csv")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 300)

In [24]:
# First 5 rows
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2024,SE,FT,AI Engineer,202730,USD,202730,US,0,US,M
1,2024,SE,FT,AI Engineer,92118,USD,92118,US,0,US,M
2,2024,SE,FT,Data Engineer,130500,USD,130500,US,0,US,M
3,2024,SE,FT,Data Engineer,96000,USD,96000,US,0,US,M
4,2024,SE,FT,Machine Learning Engineer,190000,USD,190000,US,0,US,M


In [25]:
# Optimizing the data
df = df[["work_year", "experience_level", "employment_type", "job_title", "salary_in_usd", "employee_residence", "remote_ratio", "company_location", "company_size"]]

df[["experience_level", "employment_type", "job_title", "employee_residence", "company_location", "company_size"]] = df[
    ["experience_level", "employment_type", "job_title", "employee_residence", "company_location", "company_size"]
].astype("category")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16534 entries, 0 to 16533
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   work_year           16534 non-null  int64   
 1   experience_level    16534 non-null  category
 2   employment_type     16534 non-null  category
 3   job_title           16534 non-null  category
 4   salary_in_usd       16534 non-null  int64   
 5   employee_residence  16534 non-null  category
 6   remote_ratio        16534 non-null  int64   
 7   company_location    16534 non-null  category
 8   company_size        16534 non-null  category
dtypes: category(6), int64(3)
memory usage: 511.9 KB


In [26]:
# Checking for nulls
df.isna().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [27]:
# Checking for duplicates
df.duplicated().sum()

6422

In [28]:
# To check the data types of the columns
df.dtypes

work_year                int64
experience_level      category
employment_type       category
job_title             category
salary_in_usd            int64
employee_residence    category
remote_ratio             int64
company_location      category
company_size          category
dtype: object